In [79]:
import pandas as pd
import numpy as np
import json

In [80]:
pd.set_option('display.max_columns', None)

In [81]:
def process_cbp_metadata(file_path):
    cbp_dict = {}
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)
        for row in csv_reader:
            if len(row) == 2:
                cbp_dict[row[0]] = row[1]
    return cbp_dict

def process_year_data(year, secondary_year):
    file_path = f'CBP{year}.CB{secondary_year}00CBP-Data.csv'
    meta_data_file_path = 'CBP2018.CB1800CBP-Column-Metadata.csv'

    df = pd.read_csv(file_path)
    cbp_dict = process_cbp_metadata(meta_data_file_path)

    df = df.rename(columns=cbp_dict)

    # filtered_columns = [
    #     col for col in df.columns
    #     if "Establishments" in col or "Geography" in col or "Geographic Area Name" in col
    # ]

    # df = df[filtered_columns]
    df = df.iloc[1:]  # Remove the first row if it contains column descriptions

    # Add a year column
    # df['Year'] = year

    return df

# Process data for years 2018 to 2022
years = range(2018, 2023)
secondary_years = range(18, 23)
zipped_years = dict(zip(years, secondary_years))
dfs = []

for year, secondary_year in zipped_years.items():
    try:
        df = process_year_data(year, secondary_year)
        dfs.append(df)
        print(f"Successfully processed data for {year}")
    except FileNotFoundError:
        print(f"File for year {year} not found. Skipping.")
    except Exception as e:
        print(f"Error processing data for {year}: {str(e)}")

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined dataframe
print("\nFirst few rows of the combined dataframe:")
display(combined_df.head())

# Print some information about the combined dataset
# print(f"\nTotal number of rows: {len(combined_df)}")
# print(f"Years included: {combined_df['Year'].unique()}")
# print(f"\nColumns in the dataset:")
# for col in combined_df.columns:
#     print(col)

# combined_df.to_csv('combined_cbp_data_2018_2022.csv', index=False)

<ipython-input-81-159a99f8c11c>:15: DtypeWarning: Columns (4,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully processed data for 2018


<ipython-input-81-159a99f8c11c>:15: DtypeWarning: Columns (4,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully processed data for 2019


<ipython-input-81-159a99f8c11c>:15: DtypeWarning: Columns (4,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


...

<ipython-input-81-159a99f8c11c>:15: DtypeWarning: Columns (4,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully processed data for 2021


<ipython-input-81-159a99f8c11c>:15: DtypeWarning: Columns (4,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully processed data for 2022

First few rows of the combined dataframe:


,Geographic identifier code,Geographic Area Name,2017 NAICS code,Meaning of NAICS code,Legal form of organization code,Meaning of Legal form of organization code,Employment size of establishments code,Meaning of Employment size of establishments code,Year,Number of establishments,"Annual payroll ($1,000)",Noise range for annual payroll,"First-quarter payroll ($1,000)",Noise range for first-quarter payroll,Number of employees,Noise range for number of employees,Unnamed: 16
0,0500000US01001,"Autauga County, Alabama",00,Total for all sectors,001,All establishments,001,All establishments,2018,855,373865,G,90886,G,11397,G,NaN
1,0500000US01001,"Autauga County, Alabama",00,Total for all sectors,001,All establishments,210,Establishments with less than 5 employees,2018,398,N,N,N,N,N,N,NaN
2,0500000US01001,"Autauga County, Alabama",00,Total for all sectors,001,All establishments,220,Establishments with 5 to 9 employees,2018,200,N,N,N,N,N,N,NaN
3,0500000US01001,"Autauga County, Alabama",00,Total for all sectors,001,All establishments,230,Establishments with 10 to 19 employees,2018,115,N,N,N,N,N,N,NaN
4,0500000US01001,"Autauga County, Alabama",00,Total for all sectors,001,All establishments,241,Establishments with 20 to 49 employees,2018,91,N,N,N,N,N,N,NaN


In [82]:
combined_df = combined_df.loc[:, ~combined_df.columns.str.contains('Noise', case=False)]

In [83]:
combined_df.to_csv('combined_cbp_data_2018_2022.csv', index=False)

In [84]:
zips = pd.read_csv('uszips.xlsx - Sheet1.csv')

In [85]:
zips.to_csv('zips_to_counties.csv', index=False)

Geographic Standardization

In [86]:
def standardize_geography(cbp_df, zip_county_df):
    # Convert county FIPS to string and ensure it's 5 digits long
    cbp_df['county_fips'] = cbp_df['Geographic identifier code'].str[-5:].str.zfill(5)

    # Function to get the majority county
    def get_majority_county(county_weights):
        weights = json.loads(county_weights.replace("'", '"'))
        return max(weights, key=weights.get)

    # Apply the function to get the majority county for each zip code
    zip_county_df['majority_county_fips'] = zip_county_df['county_weights'].apply(get_majority_county)

    zip_county_df['zip'] = pd.to_numeric(zip_county_df['zip'], errors='coerce')
    zip_county_df['zip'] = zip_county_df['zip'].astype('Int64')

    # Merge CBP data with zip code mapping
    merged_df = pd.merge(cbp_df, zip_county_df[['zip', 'majority_county_fips']],
                         left_on='county_fips', right_on='majority_county_fips', how='left')

    return merged_df

cbp_df = pd.read_csv('combined_cbp_data_2018_2022.csv')
zip_county_df = pd.read_csv('zips_to_counties.csv')

# Apply the function
standardized_df = standardize_geography(cbp_df, zip_county_df)
# standardized_df = standardized_df.dropna()

# print(standardized_df.head())
print("\nUnique zip codes mapped:", standardized_df['zip'].nunique())
print("Total rows in standardized data:", len(standardized_df))


Unique zip codes mapped: 33768
Total rows in standardized data: 16380191


In [87]:
standardized_df = standardized_df.drop(columns=['Unnamed: 16'])

In [88]:
standardized_df = standardized_df.dropna(subset=['zip', 'majority_county_fips'])
# .to_csv('standardized_df.csv')

In [89]:
standardized_df = standardized_df.drop(columns=['Annual payroll ($1,000)',	'First-quarter payroll ($1,000)',	'Number of employees'])

In [90]:
standardized_df.sample(10).to_csv('standardized_df.csv')

In [91]:
standardized_df[standardized_df['Meaning of Legal form of organization code'] == 'All establishments'].shape

(16370510, 13)

In [92]:
standardized_df = standardized_df.drop(columns=['Legal form of organization code',	'Meaning of Legal form of organization code'])

In [93]:
standardized_df['Meaning of Employment size of establishments code'].unique()

array(['All establishments', 'Establishments with less than 5 employees',
       'Establishments with 5 to 9 employees',
       'Establishments with 10 to 19 employees',
       'Establishments with 20 to 49 employees',
       'Establishments with 50 to 99 employees',
       'Establishments with 100 to 249 employees',
       'Establishments with 250 to 499 employees',
       'Establishments with 500 to 999 employees',
       'Establishments with 1,000 employees or more',
       'Establishments with 1,000 to 1,499 employees',
       'Establishments with 1,500 to 2,499 employees',
       'Establishments with 2,500 to 4,999 employees',
       'Establishments with 5,000 employees or more'], dtype=object)

In [94]:
large_establishment_sizes = [
        'Establishments with 100 to 249 employees',
        'Establishments with 250 to 499 employees',
        'Establishments with 500 to 999 employees',
        'Establishments with 1,000 employees or more',
        'Establishments with 1,000 to 1,499 employees',
        'Establishments with 1,500 to 2,499 employees',
        'Establishments with 2,500 to 4,999 employees',
        'Establishments with 5,000 employees or more'
    ]

standardized_df = standardized_df[standardized_df['Meaning of Employment size of establishments code'].isin(large_establishment_sizes)]

In [95]:
standardized_df.shape

(2573661, 11)

table no. 1 -
select metro region
then they select county
then zip

table no. 2 - trended timeline

In [96]:
my_list = list(pd.read_csv('Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')['RegionName'].unique())


In [97]:
with open('regions_list.txt', 'w') as file:
    # Write each item in the list to the file
    for item in my_list:
        file.write(f"{item}\n")


In [98]:
def create_region_mapping(zip_county_df, regions_list):
    # Create a dictionary of regions (excluding 'United States')
    regions_dict = {region.lower(): region for region in regions_list[1:]}

    # Create a mapping of zip codes to regions
    zip_to_region = {}
    for _, row in zip_county_df.iterrows():
        city_state = f"{row['city']}, {row['state_id']}".lower()
        region = regions_dict.get(city_state, 'Other')
        zip_to_region[row['zip']] = region

    return zip_to_region

def map_zips_to_regions(standardized_df, zip_to_region):
    # Map regions to standardized_df
    standardized_df['Region'] = standardized_df['zip'].map(zip_to_region)

    # Fill any remaining NaN values with 'Other'
    standardized_df['Region'] = standardized_df['Region'].fillna('Other')

    return standardized_df

# standardized_df = pd.read_csv('standardized_df.csv')  # Load your standardized dataframe
# zip_county_df = pd.read_csv('zips_to_counties_head.csv')

# Load the regions list
with open('regions_list.txt', 'r') as f:
    regions_list = f.read().splitlines()

# Create the region mapping
zip_to_region = create_region_mapping(zip_county_df, regions_list)

# Apply the mapping to standardized_df
final_df = map_zips_to_regions(standardized_df, zip_to_region)

display(final_df[['Geographic Area Name', 'majority_county_fips', 'zip', 'Region']].head(10))
display("\nUnique regions:", final_df['Region'].nunique())
display("Rows with 'Other' as region:", (final_df['Region'] == 'Other').sum())

# Check for any zip codes that didn't match a region
unmatched_zips = final_df[final_df['Region'] == 'Other'][['zip', 'Geographic Area Name']].drop_duplicates()
display("\nSample of unmatched zip codes:")
display(unmatched_zips.head(10))  # Print first 10 unmatched zip codes

,Geographic Area Name,majority_county_fips,zip,Region
42,"Autauga County, Alabama",01001,36003,Other
43,"Autauga County, Alabama",01001,36006,Other
44,"Autauga County, Alabama",01001,36008,Other
45,"Autauga County, Alabama",01001,36051,Other
46,"Autauga County, Alabama",01001,36066,Other
47,"Autauga County, Alabama",01001,36067,Other
48,"Autauga County, Alabama",01001,36749,Other
231,"Autauga County, Alabama",01001,36003,Other
232,"Autauga County, Alabama",01001,36006,Other
233,"Autauga County, Alabama",01001,36008,Other


'\nUnique regions:'

871

"Rows with 'Other' as region:"

1828951

'\nSample of unmatched zip codes:'

,zip,Geographic Area Name
42,36003,"Autauga County, Alabama"
43,36006,"Autauga County, Alabama"
44,36008,"Autauga County, Alabama"
45,36051,"Autauga County, Alabama"
46,36066,"Autauga County, Alabama"
47,36067,"Autauga County, Alabama"
48,36749,"Autauga County, Alabama"
656,36507,"Baldwin County, Alabama"
657,36511,"Baldwin County, Alabama"
660,36530,"Baldwin County, Alabama"


In [99]:
def calculate_economic_diversity_index(df):
    if '2017 NAICS code' in df.columns:
        # Group by Geographic Area Name and NAICS code, then count establishments
        diversity_df = df.groupby(['Geographic Area Name', '2017 NAICS code'])['Number of establishments'].sum().unstack(fill_value=0)

        # Calculate the proportion of establishments in each NAICS code
        diversity_df = diversity_df.div(diversity_df.sum(axis=1), axis=0)

        # Calculate the economic diversity index
        def economic_diversity(x):
            x = x[x > 0]  # Remove zero values
            return -np.sum(x * np.log(x))

        economic_diversity_index = diversity_df.apply(economic_diversity, axis=1)

        return economic_diversity_index
    else:
        return None


def engineer_features(df):
    # Ensure all relevant columns are numeric
    numeric_columns = ['Number of establishments']
    # , 'Annual payroll ($1,000)', 'Number of employees']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 1. Business Density: Number of establishments per 1000 people
    # df['business_density'] = (df['Number of establishments'] / df['Number of employees']) * 1000

    # # 2. Average Annual Payroll per Employee
    # df['avg_annual_payroll_per_employee'] = (df['Annual payroll ($1,000)'] * 1000) / df['Number of employees']

    # # 3. Average Establishment Size
    # df['avg_establishment_size'] = df['Number of employees'] / df['Number of establishments']

    # # 4. Payroll per Establishment
    # df['payroll_per_establishment'] = df['Annual payroll ($1,000)'] / df['Number of establishments']

    # 5. Year-over-Year Growth Rates (for multi-year data)
    if 'Year' in df.columns:
        for metric in ['Number of establishments']:
        # , 'Annual payroll ($1,000)', 'Number of employees']:
            # df[f'{metric}_growth'] = df.groupby('zip')[metric].pct_change()

        # Calculate year-over-year growth for Number of establishments
          # df['Establishment_YOY_Growth'] = df.groupby('zip')['Number of establishments'].pct_change()

          # # Calculate 5-year growth rate
          # df['Establishment_Five_Year_Growth'] = df.groupby('zip')['Number of establishments'].transform(
          #     lambda x: (x.iloc[-1] / x.iloc[0]) ** (1/5) - 1 if len(x) >= 5 else np.nan
          # )
           # Sort the dataframe by zip and year
          df = df.sort_values(['zip', 'Year'])

          # Calculate year-over-year change
          df['Establishment_YoY_Change'] = df.groupby('zip')['Number of establishments'].pct_change()

          # Calculate average annual growth rate over 5 years
          # def calc_cagr(group):
          #     years = group['Year'].nunique()
          #     if years >= 5:
          #         start_value = group.iloc[0]['Number of establishments']
          #         end_value = group.iloc[-1]['Number of establishments']
          #         cagr = (end_value / start_value) ** (1/years) - 1
          #         return cagr
          #     else:
          #         return np.nan

          # df['Five_Year_Avg_Growth'] = df.groupby('zip').apply(calc_cagr).reset_index(level=0, drop=True)



    # 6. Economic Diversity Index (using establishment counts across different NAICS codes)
    # if '2017 NAICS code' in df.columns:
    #     diversity_df = df.groupby(['Geographic Area Name', '2017 NAICS code'])['Number of establishments'].sum().unstack()
    #     diversity_df = diversity_df.div(diversity_df.sum(axis=1), axis=0)
    #     df['economic_diversity_index'] = diversity_df.apply(lambda x: -np.sum(x * np.log(x) if x > 0 else 0), axis=1)
    economic_diversity_index = calculate_economic_diversity_index(df)
    if economic_diversity_index is not None:
        df = df.merge(economic_diversity_index.rename('economic_diversity_index'),
                      left_on='Geographic Area Name',
                      right_index=True,
                      how='left')

    # 7. Small Business Ratio (assuming establishments with < 20 employees are small)
    # small_biz = df[df['Employment size of establishments code'].isin(['210', '220', '230'])].groupby('Geographic Area Name')['Number of establishments'].sum()
    # total_biz = df.groupby('Geographic Area Name')['Number of establishments'].sum()
    # df['small_business_ratio'] = df['Geographic Area Name'].map(small_biz / total_biz)

    return df

# Apply feature engineering
df_engineered = engineer_features(final_df)

# Display the first few rows of the new features
# new_features = ['business_density', 'avg_annual_payroll_per_employee', 'avg_establishment_size',
#                 'payroll_per_establishment', 'small_business_ratio']
# if 'Year' in final_df.columns:
#     new_features.extend(['Number of establishments_growth', 'Annual payroll ($1,000)_growth', 'Number of employees_growth'])
# if 'economic_diversity_index' in df_engineered.columns:
#     new_features.append('economic_diversity_index')


In [100]:
df_engineered['economic_diversity_index'].unique()

array([0.85246248, 0.66627844, 0.85424343, ..., 0.899409  , 0.99701893,
       0.58851968])

In [101]:
industry_diversity = df_engineered.groupby('Meaning of NAICS code')['economic_diversity_index'].agg(['mean', 'count']).reset_index()

# Sort by mean diversity index in descending order
industry_diversity = industry_diversity.sort_values('mean', ascending=True)

# Display top 10 industries with highest average economic diversity
print("Top 10 Industries with Highest Average Economic Diversity:")
display(industry_diversity.head(10))

Top 10 Industries with Highest Average Economic Diversity:


,Meaning of NAICS code,mean,count
16,Total for all sectors,1.482888,467901
10,Manufacturing,1.605219,226488
15,Retail trade,1.643579,169638
7,Health care and social assistance,1.699193,270935
11,"Mining, quarrying, and oil and gas extraction",1.755541,10158
0,Accommodation and food services,1.779113,103847
2,"Agriculture, forestry, fishing and hunting",1.786584,1659
1,Administrative and support and waste managemen...,1.810110,159455
17,Transportation and warehousing,1.827929,143872
4,Construction,1.836689,110680


In [104]:
df_engineered.sample(10)

,Geographic identifier code,Geographic Area Name,2017 NAICS code,Meaning of NAICS code,Employment size of establishments code,Meaning of Employment size of establishments code,Year,Number of establishments,county_fips,zip,majority_county_fips,Region,Establishment_YoY_Change,economic_diversity_index
12554455,0500000US48113,"Dallas County, Texas",81,Other services (except public administration),254,Establishments with 500 to 999 employees,2021,3,48113,75052,48113,Other,-0.812500,2.026154
13464338,0500000US06093,"Siskiyou County, California",00,Total for all sectors,251,Establishments with 100 to 249 employees,2022,7,06093,96038,06093,Other,1.333333,0.533164
15602549,0500000US42071,"Lancaster County, Pennsylvania",31-33,Manufacturing,252,Establishments with 250 to 499 employees,2022,16,42071,17603,42071,"Lancaster, PA",-0.750000,1.791532
10268032,0500000US08123,"Weld County, Colorado",48-49,Transportation and warehousing,251,Establishments with 100 to 249 employees,2021,4,08123,80534,08123,Other,-0.200000,1.559510
6883870,0500000US06085,"Santa Clara County, California",55,Management of companies and enterprises,262,"Establishments with 1,000 to 1,499 employees",2020,3,06085,95013,06085,Other,-0.750000,1.915134
2159469,0500000US39035,"Cuyahoga County, Ohio",61,Educational services,252,Establishments with 250 to 499 employees,2018,6,39035,44123,39035,Other,-0.666667,1.933531
7214924,0500000US16055,"Kootenai County, Idaho",44-45,Retail trade,251,Establishments with 100 to 249 employees,2020,14,16055,83854,16055,Other,3.666667,1.421521
2575043,0500000US44007,"Providence County, Rhode Island",54,"Professional, scientific, and technical services",251,Establishments with 100 to 249 employees,2018,9,44007,2861,44007,Other,1.250000,1.822425
10100346,0500000US06065,"Riverside County, California",23,Construction,260,"Establishments with 1,000 employees or more",2021,3,06065,92543,06065,Other,-0.700000,1.854995
5340926,0500000US37183,"Wake County, North Carolina",71,"Arts, entertainment, and recreation",251,Establishments with 100 to 249 employees,2019,23,37183,27605,37183,"Raleigh, NC",4.750000,1.949883


In [105]:
# df_engineered
df_engineered = df_engineered.rename(columns={'Geographic Area Name': 'County', 'Region': 'Metro Region'})


In [107]:
# df_engineered['county_fips'].equals(df_engineered['majority_county_fips'])

True

In [109]:
df_engineered = df_engineered.drop(columns=['county_fips'])

In [110]:
df_engineered.head()

,Geographic identifier code,County,2017 NAICS code,Meaning of NAICS code,Employment size of establishments code,Meaning of Employment size of establishments code,Year,Number of establishments,zip,majority_county_fips,Metro Region,Establishment_YoY_Change,economic_diversity_index
3225940,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,251,Establishments with 100 to 249 employees,2018,15,603,72005,Other,NaN,0.852462
3225942,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,252,Establishments with 250 to 499 employees,2018,4,603,72005,Other,-0.733333,0.852462
3225944,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,254,Establishments with 500 to 999 employees,2018,4,603,72005,Other,0.000000,0.852462
3225980,0500000US72005,"Aguadilla Municipio, Puerto Rico",44-45,Retail trade,251,Establishments with 100 to 249 employees,2018,4,603,72005,Other,0.000000,0.852462
3225990,0500000US72005,"Aguadilla Municipio, Puerto Rico",48-49,Transportation and warehousing,251,Establishments with 100 to 249 employees,2018,3,603,72005,Other,-0.250000,0.852462


In [111]:
def standardize_zip_codes(df, zip_column='zip'):
    # Convert zip codes to strings
    df[zip_column] = df[zip_column].astype(str)

    # Function to pad zip codes with leading zeros
    def pad_zip(zip_code):
        try:
            return zip_code.zfill(5)
        except AttributeError:
            # Handle any non-string values
            return '00000'

    # Apply the padding function
    df[zip_column] = df[zip_column].apply(pad_zip)

    return df

final_df_engineered = standardize_zip_codes(df_engineered)


In [115]:
final_df_engineered.head()

,Geographic identifier code,County,2017 NAICS code,Meaning of NAICS code,Employment size of establishments code,Meaning of Employment size of establishments code,Year,Number of establishments,zip,majority_county_fips,Metro Region,Establishment_YoY_Change,economic_diversity_index
3225940,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,251,Establishments with 100 to 249 employees,2018,15,00603,72005,Other,NaN,0.852462
3225942,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,252,Establishments with 250 to 499 employees,2018,4,00603,72005,Other,-0.733333,0.852462
3225944,0500000US72005,"Aguadilla Municipio, Puerto Rico",00,Total for all sectors,254,Establishments with 500 to 999 employees,2018,4,00603,72005,Other,0.000000,0.852462
3225980,0500000US72005,"Aguadilla Municipio, Puerto Rico",44-45,Retail trade,251,Establishments with 100 to 249 employees,2018,4,00603,72005,Other,0.000000,0.852462
3225990,0500000US72005,"Aguadilla Municipio, Puerto Rico",48-49,Transportation and warehousing,251,Establishments with 100 to 249 employees,2018,3,00603,72005,Other,-0.250000,0.852462
